In [38]:
import pandas as pd
import numpy as np
import re

In [39]:
LITNET   = "litnet_books_dataset.csv"
LITRES   = "litres_books.csv"
AUTHODAY = "author_today.csv"

In [40]:
def norm_title(s):
    s = str(s).lower() if pd.notna(s) else ""
    s = re.sub(r"\([^)]*\)", " ", s)                 
    s = re.sub(r"\b(книга|том|часть)\s*\d+\b", " ", s)
    s = s.split(":")[0].split("–")[0]           
    s = s.replace("«"," ").replace("»"," ")
    s = re.sub(r"[^a-zа-яё0-9\s]", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def norm_author(s):
    s = str(s).lower() if pd.notna(s) else ""
    s = re.split(r"[;,/&]| и ", s)[0]  
    s = s.replace("«"," ").replace("»"," ")
    s = re.sub(r"[^a-zа-яё0-9\s\-\.]", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def ensure_author_col(df):
    for c in ["authors", "author", "Автор", "автор"]:
        if c in df.columns:
            return c
    df["authors"] = ""
    return "authors"

def coalesce(df, cols):
    cols = [c for c in cols if c in df.columns]
    if not cols: 
        return pd.Series([None]*len(df))
    return df[cols].bfill(axis=1).iloc[:, 0]

In [41]:

litnet   = pd.read_csv(LITNET)
litres   = pd.read_csv(LITRES)
authoday = pd.read_csv(AUTHODAY)

In [42]:
ln_auth = ensure_author_col(litnet)
lr_auth = ensure_author_col(litres)
ad_auth = ensure_author_col(authoday)

In [43]:
for df, a_col, t_col in [
    (litnet, ln_auth, "title"),
    (litres, lr_auth, "title"),
    (authoday, ad_auth, "title"),
]:
    df["author_norm"] = df[a_col].map(norm_author)
    df["title_norm"]  = df[t_col].map(norm_title)
    df["join_key"]    = (df["author_norm"] + " | " + df["title_norm"]).str.strip(" |")

def add_suffix(df, sfx):
    keep = {"join_key", "author_norm", "title_norm"}
    return df.rename(columns={c: (c if c in keep else f"{c}_{sfx}") for c in df.columns})

In [44]:
litnet_s   = add_suffix(litnet,   "litnet")
litres_s   = add_suffix(litres,   "litres")
authoday_s = add_suffix(authoday, "authoday")

In [45]:

m = litres_s.merge(litnet_s, on="join_key", how="outer").merge(authoday_s, on="join_key", how="outer")


m["has_litres"]   = m.filter(like="_litres").notna().any(axis=1)
m["has_litnet"]   = m.filter(like="_litnet").notna().any(axis=1)
m["has_authoday"] = m.filter(like="_authoday").notna().any(axis=1)

In [46]:
def _norm_cell(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    y = pd.to_numeric(s.replace(",", "."), errors="coerce")
    if pd.notna(y): return float(y)
    return re.sub(r"\s+", " ", s).lower()

def strict_unify(df, out, cols3):
    cols3 = [c for c in cols3 if c in df.columns]
    if len(cols3) < 3:
        df[out] = np.nan
        df[f"{out}_equal_all"] = False
        return df
    a,b,c = (df[cols3[0]].map(_norm_cell),
             df[cols3[1]].map(_norm_cell),
             df[cols3[2]].map(_norm_cell))
    ok = a.notna() & b.notna() & c.notna() & (a==b) & (b==c)
    df[out] = np.where(ok, df[cols3[0]], np.nan)
    df[f"{out}_equal_all"] = ok
    return df

FAMILIES = {
    "title":    ["title_litres","title_authoday","title_litnet"],
    "authors":  ["authors_litres","authors_authoday","author_litnet"],
    "price":    ["price_litres","price_authoday","price_litnet"],
    "date":     ["release_date_litres","date_authoday","year_litnet"],
    "rate":     ["rating_litres","rate_litnet","rate_litnet"],
    "categories":["genres_litres","categories_authoday","genre_litnet"],
    "description":["description_litres","annotation_authoday","description_litnet"],
    "age":      ["age_limit_litres","age_litnet","age_litnet"],
}

for out, cols in FAMILIES.items():
    m = strict_unify(m, out, cols)

audit_cols = [f"{k}_equal_all" for k in FAMILIES]
m_conflicts = m.loc[~m[audit_cols].all(axis=1), ["join_key"] + audit_cols]
print("Совпали у всех три источника по полям (в %):")
print((m[audit_cols].mean()*100).round(2).astype(str) + "%")

m[["join_key"] + list(FAMILIES.keys())].to_csv("strict_view_only_equal.csv", index=False)
m_conflicts.to_csv("strict_conflicts_report.csv", index=False)

Совпали у всех три источника по полям (в %):
title_equal_all          0.01%
authors_equal_all        0.02%
price_equal_all           0.0%
date_equal_all            0.0%
rate_equal_all            0.0%
categories_equal_all      0.0%
description_equal_all     0.0%
age_equal_all             0.0%
dtype: object


In [47]:
front = ["join_key","has_litres","has_litnet","has_authoday",
         "title","authors","price","date","views","likes","comments","rate",
         "author_norm_litres","title_norm_litres",
         "author_norm_litnet","title_norm_litnet",
         "author_norm_authoday","title_norm_authoday"]
front = [c for c in front if c in m.columns]
m = m[front + [c for c in m.columns if c not in front]]


In [48]:

m.to_csv("books_full_outer_merged.csv", index=False)


In [49]:
m.drop(columns=['price', 'date', 'rate'])
m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25375 entries, 0 to 25374
Data columns (total 66 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   join_key                 25375 non-null  object 
 1   has_litres               25375 non-null  bool   
 2   has_litnet               25375 non-null  bool   
 3   has_authoday             25375 non-null  bool   
 4   title                    3 non-null      object 
 5   authors                  4 non-null      object 
 6   price                    0 non-null      float64
 7   date                     0 non-null      object 
 8   rate                     0 non-null      float64
 9   url_litres               14063 non-null  object 
 10  title_litres             14063 non-null  object 
 11  authors_litres           14062 non-null  object 
 12  rating_litres            13574 non-null  float64
 13  rating_count_litres      13574 non-null  float64
 14  reviews_count_litres  

In [50]:
m.columns

Index(['join_key', 'has_litres', 'has_litnet', 'has_authoday', 'title',
       'authors', 'price', 'date', 'rate', 'url_litres', 'title_litres',
       'authors_litres', 'rating_litres', 'rating_count_litres',
       'reviews_count_litres', 'price_litres', 'genres_litres',
       'age_limit_litres', 'release_date_litres', 'written_date_litres',
       'pages_litres', 'isbn_litres', 'copyright_holder_litres',
       'formats_litres', 'description_litres', 'author_norm_x', 'title_norm_x',
       'id_litnet', 'available_litnet', 'rate_litnet', 'title_litnet',
       'author_litnet', 'genre_litnet', 'price_litnet', 'year_litnet',
       'description_litnet', 'age_litnet', 'author_norm_y', 'title_norm_y',
       'title_authoday', 'authors_authoday', 'categories_authoday',
       'date_authoday', 'symbols_count_authoday', 'a4_sheets_authoday',
       'views_authoday', 'likes_authoday', 'comments_authoday',
       'reviews_authoday', 'price_authoday', 'cycle_authoday',
       'exclusive_autho

In [51]:
m.to_csv("books_after_merge.csv", index=False)